# Minimum k-cut Algorithm

### High-level Algorithm Specification:

1. Create vertex list and an edges list, e.g.:

    ```javascript
    vertices = {1: [2,4,5], 2: [3,4,5], 3: [2,4], 4: [1,2,3], 5: [1,2]}
    edges = [[1,2], [1,4], [1,5], [2,3], [2,4], [2,5], [3,4]]
    ```

2. Keep track of the minimum cut so far:

    ```javascript
    // really this could be the max degree of all vertices, I believe
    min_edges_so_far = len(edges)
    min_vertex_sets = {1:[], 2:[], 3:[], 4:[], 5:[]}
    ```

3. *Iterate at least `n^2 log n` times (where n is the original number of vertices)*
    
    **Intiate:**
    
    ```javascript
    temp_vertex_sets = copy(min_vertex_sets)
    temp_vertices = copy(vertices)
    temp_edges = copy(edges)
    ```

    **While num_vertices > k:**

    1. Pick an edge at random: the first vertex (`v1`) will absorb the second (`v2`). Add `v2` and `temp_vertex_sets[v2]` to `temp_vertex_sets[v1]` and delete `temp_vertex_sets[v2]`.
    2. All vertices adjacent to `v2` are added to `temp_vertices[v1]` unless already present. Remove `v2` from `temp_vertices[v1]`.
    3. Replace all instances of `v2` in `temp_edges` with `v1`, unless the other vertex of the edge is itself `v1`. In the latter case, delete the edge (e.g. remove self-loops). **Note:** Parallel edges are allowed; there may be multiple instances of an edge comprised the same vertex pair.

    **Finally:** The number of final edges is the number of edges across the final cut in this iteration. If it is less than min_edges_so_far, update `min_edges_so_far = len(temp_edges)` and `min_vertex_sets = temp_vertex_sets`.
        

## Setup: Select all measurements and document ids from the database

In [1]:
import psycopg2
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import numpy as np
import json
from copy import deepcopy
import random

database = 'fomc'
conn = psycopg2.connect("dbname=" + database + " user=abarciauskas")
cur = conn.cursor()

year = 2006
cosine_thresh = 0.25
limit = 1000
cur.execute("SELECT Doc1Id,Doc2Id,CosineSimilarity FROM alignments WHERE Year = '" + str(year) + "'"
           " AND CosineSimilarity >= " + str(cosine_thresh) + " ORDER BY random() LIMIT " + str(limit))
cosine_sims = cur.fetchall()
len(cosine_sims)

1000

## Step 1: Create the graph

The graph is comprised a list of edges (a vertex tuple) and a dictionary of vertices.

In [2]:
def create_graph(alignments):
    edges = [tuple([x[0],x[1]]) for x in alignments]
    vertices = {}
    for edge in edges:
        v1 = edge[0]
        v2 = edge[1]
        if v1 in vertices.keys():
            vertices[v1].add(v2)
        else:
            vertices[v1] = {v2}
        if v2 in vertices.keys():
            vertices[v2].add(v1)
        else:
            vertices[v2] = {v1}
    return [edges, vertices]

edges, vertices = create_graph(cosine_sims)
print len(vertices)
print len(edges)

785
1000


In [3]:
# need to find disconnected graphs
graphs = []
unvisited = set(vertices.keys())

#Detect distinct graphs
while len(unvisited) > 0:
    # for every vertex, find all of its connected components and recurse on those vertices
    visited = []
    current_vertex = unvisited.pop()
    visited.append(current_vertex)
    stack_to_visit = list(vertices[current_vertex])
    while len(stack_to_visit) > 0:
        current_vertex = stack_to_visit.pop()
        current_adj_vtcs = vertices[current_vertex]
        if current_vertex not in visited: visited.append(current_vertex)
        if current_vertex in unvisited: unvisited.remove(current_vertex)        
        for v in current_adj_vtcs:
            if v not in visited:
                stack_to_visit.insert(0, v)
    graphs.append(visited)

# print len(unvisited)
# print len(visited)
# print len(graphs)
# print ''

graph_lengths = [len(graph) for graph in graphs]
fc_graph = graphs[graph_lengths.index(max(graph_lengths))]
print 'Number vertices fully connected graph: ' + str(len(fc_graph))

Number vertices fully connected graph: 524


In [4]:
# Remove loner graphs
# fc = fully connected
set_fc_graph_vertices = set(fc_graph)
loners = set_fc_graph_vertices ^ set(vertices.keys())

fc_vertices = deepcopy(vertices)
fc_edges = deepcopy(edges)

print(len(fc_vertices))
for loner in loners:
    fc_vertices.pop(loner, None)
print(len(fc_vertices))

print(len(fc_edges))
fc_edges = filter(lambda x: not list(x)[0] in loners and not list(x)[1] in loners, fc_edges)    
print(len(fc_edges))

785
524
1000
738


## Step 2: Keep track of minimum so far

## Step 3: Random iterations

In [5]:
import random
def run(niters):
    min_fc_edges_so_far = len(fc_edges)
    min_vertex_sets = {key: set() for key in fc_vertices.keys()}
    for iteridx in range(niters):
        if iteridx % 50 == 0: print 'Running iter: ' + str(iteridx)        
        temp_vertex_sets = {key: set() for key in fc_vertices.keys()}
        temp_fc_vertices = deepcopy(fc_vertices)
        temp_fc_edges = fc_edges[:]        
        while len(temp_fc_vertices) > k:
            # pick an edge at random and delete it
            rand_idx = int(random.random()*len(temp_fc_edges))
            random_edge = temp_fc_edges.pop(rand_idx)
            # Add v2 and temp_vertex_sets[v2] to temp_vertex_sets[v1] and delete temp_vertex_sets[v2]
            v1 = list(random_edge)[0]
            v2 = list(random_edge)[1]
            temp_vertex_sets[v1] = temp_vertex_sets[v1].union(temp_vertex_sets[v2])
            temp_vertex_sets[v1].add(v2)
            temp_vertex_sets.pop(v2, None)

            # All fc_vertices adjacent to v2 are added to temp_fc_vertices[v1] unless already present.
            # Remove v2 from temp_fc_vertices[v1].
            adj_v2 = temp_fc_vertices[v2]
            temp_fc_vertices[v1] = temp_fc_vertices[v1].union(adj_v2)
            temp_fc_vertices[v1].remove(v2)
            temp_fc_vertices.pop(v2, None)

            # Replace all instances of v2 in temp_fc_edges with v1, unless the other vertex of the edge is itself v1.
            # In the latter case, delete the edge (e.g. remove self-loops).
            # Note: Parallel fc_edges are allowed; there may be multiple instances of an edge comprised the same vertex pair.
            remove_fc_edges = []
            for i,cur_edge in enumerate(temp_fc_edges):
                if len(cur_edge) > 1:
                    cur_edge_v1 = list(cur_edge)[0]
                    cur_edge_v2 = list(cur_edge)[1]
                    if (cur_edge == random_edge):
                        remove_fc_edges.append(i)
                    elif cur_edge_v1 == v2:
                        temp_fc_edges[i] = {v1, cur_edge_v2}
                        # remove this edge from temp_fc_vertices
                        # it may have already been removed because we keep parallel fc_edges around
                        if v2 in temp_fc_vertices[cur_edge_v2]: temp_fc_vertices[cur_edge_v2].remove(v2)
                    elif cur_edge_v2 == v2:
                        temp_fc_edges[i] = {cur_edge_v1, v1}
                        # it may have already been removed because we keep parallel fc_edges around
                        if v2 in temp_fc_vertices[cur_edge_v1]: temp_fc_vertices[cur_edge_v1].remove(v2)
            # work around for delete
            temp_fc_edges = [set(i) for j, i in enumerate(temp_fc_edges) if j not in remove_fc_edges]
            #Finally: The number of final fc_edges is the number of fc_edges across the final cut in this iteration.
            #If it is less than min_fc_edges_so_far, update min_fc_edges_so_far = len(temp_fc_edges) and min_vertex_sets = temp_vertex_sets.
            if len(temp_fc_edges) < min_fc_edges_so_far:
                min_fc_edges_so_far = len(temp_fc_edges)
                min_vertex_sets = temp_vertex_sets
    return min_fc_edges_so_far, min_vertex_sets

In [6]:
k = 10
n = len(fc_vertices)
niters = 1

import time

t0 = time.time()
min_fc_edges_so_far, min_vertex_sets = run(niters)
t1 = time.time()

total = t1-t0
print 'Total time for ' + str(niters) + ': ' + str(total)

Running iter: 0
Total time for 1: 0.288903951645


In [8]:
niters = n**2#int(np.ceil(n**2*np.log(n)))
niters = 10000
total_seconds = niters*total
minutes = total_seconds/60
hours = minutes/60
print hours

0.802510976791


In [10]:
niters = 10000
t0 = time.time()
min_edges_so_far, min_vertex_sets = run(niters)
t1 = time.time()
total = t1-t0
print 'Total time for ' + str(niters) + ': ' + str(total)

Total time for 10000: 2612.97274208


In [17]:
print 'Num crossing edges: ' + str(min_edges_so_far)
total = t1-t0
print 'Total time for ' + str(niters) + ' iterations: ' + str(total/60/60)
super_nodes = min_vertex_sets.keys()
super_nodes = filter(lambda x: len(min_vertex_sets[x]) > 5, super_nodes)
print len(super_nodes)

Num crossing edges: 9
Total time for 10000 iterations: 0.725825761689
3
